In [1]:
import pytorch_lightning as pl
import numpy as np
import sys  
sys.path.insert(0, '../src/')

import torch
from data.har import HARDataModule
from model.classfier import HARLSTMClassfier, HARBiLSTMClassfier

In [2]:
trainer = pl.Trainer()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [3]:
with open("../data/processed/test.npz", "rb") as f:
    npzfile = np.load(f)
    X_test = torch.from_numpy(npzfile["X"]).float()
    y_test = torch.from_numpy(npzfile["y"]).int()

In [4]:
y_pred = np.zeros((y_test.shape[0], 2))

## LSTM Model

In [5]:
model = HARLSTMClassfier.load_from_checkpoint('../model/har-lstm-epoch=119-val_loss=0.99.ckpt')

In [6]:
model.eval()
outputs = model(X_test)
_, preds = torch.max(outputs, 1)
y_pred[:, 0] = preds.numpy()

## BiLSTM Model

In [7]:
model = HARBiLSTMClassfier.load_from_checkpoint('../model/har-bilstm-epoch=281-val_loss=0.56.ckpt')

In [8]:
model.eval()
outputs = model(X_test)
_, preds = torch.max(outputs, 1)
y_pred[:, 1] = preds.numpy()

In [9]:
with open('deep-learning.npy', 'wb') as f:
    np.save(f, y_pred)